# TP3 - Adrián José Zapater Reig
## Apartado 1 - Kafka

### 1. Localizar el archivo que contiene la configuración del servidor Zookeeper y levantar dicho servidor utilizando la configuración por defecto. ¿Qué puerto es el utilizado por defecto por Zookeeper?

In [1]:
# Vamos a usar el zookeeper que incorpora por defecto kafka
!cat /usr/local/kafka/config/zookeeper.properties

# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
# 
#    http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# the directory where the snapshot is stored.
dataDir=/tmp/zookeeper
# the port at which the clients will connect
clientPort=2181
# disable the per-ip limit on the number of connections since this is a non-production c

#### El puerto por defecto es 2181. Lo podemos localizar en la propiedad clientPort.

In [ ]:
# Levantamos zookeeper:
! /usr/local/kafka/bin/zookeeper-server-start.sh /usr/local/kafka/config/zookeeper.properties

## 2. Localizar el archivo que contiene la configuración de un broker y levantar dicho broker. ¿Qué puerto es el utilizado por defecto por los brokers?

In [8]:
! cat /usr/local/kafka/config/server.properties

# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# see kafka.server.KafkaConfig for additional details and defaults

############################# Server Basics #############################

# The id of the broker. This must be set to a unique integer for each broker

In [ ]:
! /usr/local/kafka/bin/kafka-server-start.sh /usr/local/kafka/config/server.properties

Puerto por defecto: 9092. Lo podemos obtener de la traza de la shell al levantar el broker.

## 3. Crear un topic nuevo con el nombre “initial”, con una partición y un factor de replicación igual a 1.

In [22]:
! /usr/local/kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic initial

In [11]:
# Comprobamos que se ha creado:
! /usr/local/kafka/bin/kafka-topics.sh --list --bootstrap-server localhost:9092

initial


## 4. Describir el topic “initial” y comentar los resultados que se obtienen.
Tenemos un topic con una única partición, por lo que dicha partición tendrá todos los mensajes.
Tenemos un único factor de replicación por lo que los mensajes solo se almacenarán en el master. No hay tolerancia a fallos.

## 5. Arrancar un productor sobre el topic “initial”.

In [ ]:
! /usr/local/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic initial

## 6. Arrancar un consumidor sobre el topic “initial” que consuma todos los mensajes desde el principio.

In [ ]:
! /usr/local/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic initial --from-beginning

## 7. Generar algunos mensajes en el productor levantado en el ejercicio 5 y observar cómo se reciben en el consumidor levantado en el ejercicio 6. Comentar la salida obtenida. Detener a continuación el productor y el consumidor.

Los mensajes que escribimos en el productor aparecen en el mismo orden en el consumidor con un pequeño delay. los mensajes que se han escrito en el productor antes de crear el consumidor también han sido recibido en el mismo orden en el consumidor.

## 8. Crear un topic nuevo con el nombre “testRep”, con dos particiones y un factor de replicación igual a 2. ¿Qué sucede? ¿Cuál es la razón?

In [ ]:
! /usr/local/kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 2 --partitions 2 --topic testRep

RESULTADO: <br/>
[2020-01-10 19:31:28,198] ERROR java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.InvalidReplicationFactorException: Replication factor: 2 larger than available brokers: 1.

Nos devuelve un error. Esto se debe a que solo tenemos un broker levantado y no puede replicar 2 veces los mensajes si solo tenemos un broker.

## 9. Detener el broker levantado en el ejercicio 2.

## 10. Levantar tres brokers distintos: generar nuevos ficheros de configuración para cada uno de ellos indicando las modificaciones que han sido necesarias. Los identificadores de los brokers serán 0, 1 y 2.

In [17]:
#Fichero de configuración de broker 1 - id 0:
! cat /usr/local/kafka/config/broker_3.properties

broker.id=0
listeners=PLAINTEXT://:9092
log.dirs=/tmp/kafka-logs-0


num.network.threads=3
num.io.threads=8

socket.send.buffer.bytes=102400
socket.receive.buffer.bytes=102400
socket.request.max.bytes=104857600


num.partitions=1
num.recovery.threads.per.data.dir=1

offsets.topic.replication.factor=1
transaction.state.log.replication.factor=1
transaction.state.log.min.isr=1

log.retention.hours=168
log.segment.bytes=1073741824
log.retention.check.interval.ms=300000

zookeeper.connect=localhost:2181
zookeeper.connection.timeout.ms=6000

group.initial.rebalance.delay.ms=0


In [18]:
#Fichero de configuración de broker 2 - id 1:
! cat /usr/local/kafka/config/broker_1.properties

broker.id=1
listeners=PLAINTEXT://:9093
log.dirs=/tmp/kafka-logs-1


num.network.threads=3
num.io.threads=8

socket.send.buffer.bytes=102400
socket.receive.buffer.bytes=102400
socket.request.max.bytes=104857600


num.partitions=1
num.recovery.threads.per.data.dir=1

offsets.topic.replication.factor=1
transaction.state.log.replication.factor=1
transaction.state.log.min.isr=1

log.retention.hours=168
log.segment.bytes=1073741824
log.retention.check.interval.ms=300000

zookeeper.connect=localhost:2181
zookeeper.connection.timeout.ms=6000

group.initial.rebalance.delay.ms=0


In [19]:
#Fichero de configuración de broker 3 - id 2:
! cat /usr/local/kafka/config/broker_2.properties

broker.id=2
listeners=PLAINTEXT://:9094
log.dirs=/tmp/kafka-logs-2


num.network.threads=3
num.io.threads=8

socket.send.buffer.bytes=102400
socket.receive.buffer.bytes=102400
socket.request.max.bytes=104857600


num.partitions=1
num.recovery.threads.per.data.dir=1

offsets.topic.replication.factor=1
transaction.state.log.replication.factor=1
transaction.state.log.min.isr=1

log.retention.hours=168
log.segment.bytes=1073741824
log.retention.check.interval.ms=300000

zookeeper.connect=localhost:2181
zookeeper.connection.timeout.ms=6000

group.initial.rebalance.delay.ms=0


In [ ]:
! /usr/local/kafka/bin/kafka-server-start.sh /usr/local/kafka/config/broker_1.properties
! /usr/local/kafka/bin/kafka-server-start.sh /usr/local/kafka/config/broker_2.properties
! /usr/local/kafka/bin/kafka-server-start.sh /usr/local/kafka/config/broker_3.properties

Para poder levantar 3 brokers necesitamos declarar cada broker con un id único, un puerto libre y un fichero de log separado.

se han modificado las siguientes propiedades:

- broker.id=<$id>

- listeners=<$puerto>

- log.dirs=<$ruta_de_logs>

## 12. Crear un topic nuevo con el nombre “testOrder1”, con 1 partición y factor de replicación igual a 3.

In [23]:
! /usr/local/kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 3 --partitions 1 --topic testOrder1
! /usr/local/kafka/bin/kafka-topics.sh --list --bootstrap-server localhost:9092

__consumer_offsets
initial
testOrder1


NOTA: Apuntamos al broker 0 como bootstrap-server para almacenar los metadatos ahí.

## 13. Crear un topic nuevo con el nombre “testOrder2”, con 3 particiones y factor de replicación igual a 3.

In [24]:
! /usr/local/kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 3 --partitions 3 --topic testOrder2
! /usr/local/kafka/bin/kafka-topics.sh --list --bootstrap-server localhost:9092

__consumer_offsets
initial
testOrder1
testOrder2


## 14. Describir los topics “testOrder1” y “testOrder2” y comentar los resultados que se obtienen.

Se han creado 2 topics, ambos con 3 factor de replicación 3 por lo que cada mensaje se almacenará 3 veces. Un topic tiene 1 partición a la que irán todos los mensajes y el otro tiene 3 particiones. Los mensajes del segundo se distribuirán por id de partición, si tiene, entre las particiones. En caso de no tener id de partición se usará el módulo de la key del mensaje y si tampoco tiene eso, se utilizará el algoritmo round robin para repartir los mensajes.

## 15. Arrancar un productor sobre el topic “testOrder1” y un consumidor desde el inicio sobre el mismo topic. Introducir algunos mensajes y observar cómo se consumen. Detener el productor y el consumidor.

In [ ]:
! /usr/local/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testOrder1

In [ ]:
! /usr/local/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic testOrder1 --from-beginning

## 16. Arrancar un productor sobre el topic “testOrder2” y un consumidor desde el inicio sobre el mismo topic. Introducir algunos mensajes y observar cómo se consumen. Detener el productor y el consumidor.

In [ ]:
! /usr/local/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testOrder2

In [ ]:
! /usr/local/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic testOrder2 --from-beginning

## 17. Arrancar un consumidor sobre el topic “testOrder1”, sin que consuma desde el principio, únicamente los valores que le lleguen desde el nuevo arranque. En otra consola, generar una secuencia de números del 1 al 30 (utilizar el comando “seq”) y enviársela mediante una tubería o pipe (indicada con el carácter “|”) a un productor sobre el topic “testOrder1”. Observar cómo se consume dicha secuencia en el consumidor.

In [ ]:
! /usr/local/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic testOrder1

In [ ]:
! seq 1 30 | /usr/local/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testOrder1

## 18. Arrancar un consumidor sobre el topic “testOrder2”, sin que consuma desde el principio, únicamente los valores que le lleguen desde el nuevo arranque. En otra consola, generar una secuencia de números del 1 al 30 (utilizar el comando “seq”) y enviársela mediante una tubería o pipe (indicada con el carácter “|”) a un productor sobre el topic “testOrder2”. Observar cómo se consume dicha secuencia en el consumidor.

In [ ]:
! /usr/local/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic testOrder2

In [ ]:
! seq 1 30 | /usr/local/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testOrder2

## 19. ¿Cuáles son las diferencias entre lo mostrado en el consumidor en los dos ejercicios anteriores? ¿A qué se deben?

Como se puede apreciar kafka sólo respeta el orden de los mensajes en una partición, pero no entre particiones. 

Se puede apreciar que, como no se ha asignado una kay para los mensajes ha usado el algoritmo round roubin para enviar un mensaje (número) a cada partición: 

1 -> partición 1, 

2 -> partición 2, 

3 -> partición 3, 

4 -> partición 1, 

....

## 20. Obtener los últimos offsets de los mensajes del topic “testOrder1” y “testOrder2”. ¿Qué se observa? (Utilizar el comando kafka-run-class con la opción kafka.tools.GetOffsetShell).

In [ ]:
! /usr/local/kafka/bin/kafka-run-class.sh kafka.tools.GetOffsetShell --broker-list localhost:9092 --topic testOrder1

In [ ]:
! /usr/local/kafka/bin/kafka-run-class.sh kafka.tools.GetOffsetShell --broker-list localhost:9092 --topic testOrder2

testOrder1 solo tiene 1 partición y por eso una única línea con 30 mensajes.

testOrder2 tiene 3 particiones y por eso tiene una línea con 10 mensajes por partición.

## 21. Detener de forma no planificada (utilizando Ctrl-C o cerrando la consola) el broker con identificador 2, y describir el topic “testOrder2”. ¿Cuáles son las diferencias con la descripción que obtuvimos en el ejercicio 14? ¿A qué se deben?

In [ ]:
! /usr/local/kafka/bin/kafka-run-class.sh kafka.tools.GetOffsetShell --broker-list localhost:9092 --topic testOrder2